In [ ]:
import numpy as np
import xarray as xr
import pandas as pd
import geoviews as gv
import geoviews.feature as gf
from geoviews import dim, opts

gv.extension('bokeh')

The Bokeh backend offers much more advanced tools to interactively explore data, making good use of GeoViews support for web mapping tile sources. As we learned in the [Projections](Projections.ipynb) user guide, using web mapping tile sources is only supported when using the default ``GOOGLE_MERCATOR`` ``crs``.

# WMTS - Tile Sources

GeoViews provides a number of tile sources by default, provided by CartoDB, Stamen, OpenStreetMap, Esri and Wikipedia. These can be imported from the ``geoviews.tile_sources`` module.

In [ ]:
import geoviews.tile_sources as gts

gv.Layout([ts.relabel(name) for name, ts in gts.tile_sources.items()]).opts(
    'WMTS', xaxis=None, yaxis=None, width=225, height=225).cols(4)

The tile sources that are defined as part of GeoViews are simply instances of the ``gv.WMTS`` and ``gv.Tiles`` elements, which accept tile source URLs of three formats:

1. Web mapping tile sources: ``{X}``, ``{Y}`` defining the location and a ``{Z}`` parameter defining the zoom level 
2. Bounding box tile source: ``{XMIN}``, ``{XMAX}``, ``{YMIN}``, and ``{YMAX}`` parameters defining the bounds
3. Quad-key tile source: a single ``{Q}`` parameter

Additional, freely available tile sources can be found at [wiki.openstreetmap.org](http://wiki.openstreetmap.org/wiki/Tile_servers).

A tile source may also be drawn at a different ``level`` allowing us to overlay a regular tile source with a set of labels. Valid options for the 'level' option include 'image', 'underlay', 'glyph', 'annotation' and 'overlay':

In [ ]:
gts.EsriImagery.opts(width=600, height=570, global_extent=True) * gts.StamenLabels.options(level='annotation')

## Plotting data

One of the main benefits of plotting data with Bokeh is the interactivity it allows. Here we will load a dataset of all the major cities in the world with their population counts over time:

In [ ]:
cities = pd.read_csv('../assets/cities.csv', encoding="ISO-8859-1")
population = gv.Dataset(cities, kdims=['City', 'Country', 'Year'])
cities.head()

Now we can convert this dataset to a set of points mapped by the latitude and longitude and containing the population, country and city as values. The longitudes and latitudes in the dataframe are supplied in simple Plate Carree coordinates, which we will need to declare (as the values are not stored with any associated units). The ``.to`` conversion interface lets us do this succinctly. Note that since we did not assign the Year dimension to the points key or value dimensions, it is automatically assigned to a HoloMap, rendering the data as an animation using a slider widget:

In [ ]:
points = population.to(gv.Points, ['Longitude', 'Latitude'], ['Population', 'City', 'Country'])
(gts.Wikipedia * points).opts(
    opts.Points(width=600, height=350, tools=['hover'], size=np.sqrt(dim('Population'))*0.005,
                color='Population', cmap='viridis'))

And because this is a fully interactive Bokeh plot, you can now hover over each datapoint to see all of the values associated with it (name, location, etc.), and you can zoom and pan using the tools provided.  Each time, the map tiles should seamlessly update to provide additional detail appropriate for that zoom level.


## Choropleths

The tutorial on [Geometries](Geometries.ipynb) covers working with shapefiles in more detail but here we will quickly combine a shapefile with a pandas DataFrame to plot the results of the EU Referendum in the UK. We begin by loading the shapefile and then us ``pd.merge`` by combining it with some CSV data containing the referendum results:

In [ ]:
import geopandas as gpd
geometries = gpd.read_file('../assets/boundaries/boundaries.shp')
referendum = pd.read_csv('../assets/referendum.csv')
gdf = gpd.GeoDataFrame(pd.merge(geometries, referendum))

Now we can easily pass the GeoDataFrame to a Polygons object and declare the ``leaveVoteshare`` as the first value dimension which it will color by:

In [ ]:
gv.Polygons(gdf, vdims=['name', 'leaveVoteshare']).opts(
    tools=['hover'], width=450, height=600, color_index='leaveVoteshare',
    colorbar=True, toolbar='above', xaxis=None, yaxis=None)

### Images

The Bokeh backend also provides basic support for working with images. In this example we will load a very simple Iris Cube and display it overlaid with the coastlines feature from Cartopy. Note that the Bokeh backend does not project the image directly into the web Mercator projection, instead relying on regridding, i.e. resampling the data using a new grid. This means the actual display may be subtly different from the more powerful image support for the matplotlib backend, which will project each of the pixels into the chosen display coordinate system without regridding.

In [ ]:
dataset = xr.open_dataset('../data/pre-industrial.nc')
air_temperature = gv.Dataset(dataset, ['longitude', 'latitude'], 'air_temperature',
                             group='Pre-industrial air temperature')
air_temperature.to.image().opts(tools=['hover'], cmap='viridis') *\
gf.coastline().opts(line_color='black', width=600, height=500)